# Characterization of stressors

The characterization of stressors is a standard procedure to calculate the environmental and social impacts of economic activity. This is usually accomplished by multiplying (matrix-multiplication) the stressor-matrix with a characterization-matrix. Doing that in the matrix forms requires a 1:1 correspondence of the columns of the characterization matrix to the rows of the stressor-matrix.

Pymrio uses a different approach with matching the strings of the
characterization table (given in long-format) to the available stressors.
Advantages:

  - enforcing correspondence of stressors across the mrio system and characterization table
  - unit checks across the extension and characterization table 
  - agnostic to the order of the entries in the characterization table
  - allows to use characterization tables which includes characterization for stressors not present in the given satellite account. All characterizations relying on not available stressor will be automatically removed.
  - efficient handling of region and/or sector specific characterization factors 
  - enables characterization of stressors across different extensions

The characterization function is available as a method of the extension object and as top-level function accepting a full MRIO object or collection of extensions (TODO: implement).

In the following, we go through some example on how to use both methods, starting with some simple example and then advancing to more complex cases with regional specific factors.

## Basic Example

For this example we use the test MRIO included in Pymrio. We also need
the Pandas library for loading the characterization table and pathlib for some folder manipulation.

In [12]:
from pathlib import Path

In [13]:
import pandas as pd

In [14]:
import pymrio
from pymrio.core.constants import PYMRIO_PATH  # noqa

To load the test MRIO we use:

In [15]:
io = pymrio.load_test()

and the characterization table with some foo factors can be loaded by

In [16]:
charact_table = pd.read_csv(
    (PYMRIO_PATH["test_mrio"] / Path("concordance") / "emissions_charact.tsv"),
    sep="\t",
)
charact_table

,stressor,compartment,impact,factor,impact_unit,stressor_unit
0,emission_type1,air,air water impact,0.002,t,kg
1,emission_type2,water,air water impact,0.001,t,kg
2,emission_type1,air,total emissions,1.000,kg,kg
3,emission_type2,water,total emissions,1.000,kg,kg
4,emission_type3,land,total emissions,1.000,kg,kg
5,emission_type1,air,total air emissions,0.001,t,kg


This table contains the columns 'stressor' and 'compartment' which correspond
to the index names of the test_mrio emission satellite accounts:

In [17]:
io.emissions.F

region                            reg1                                         \
sector                            food      mining manufactoring  electricity   
stressor       compartment                                                      
emission_type1 air          1848064.80  986448.090   23613787.00  28139100.00   
emission_type2 water         139250.47   22343.295     763569.18    273981.55   

region                                                                     \
sector                     construction      trade   transport      other   
stressor       compartment                                                  
emission_type1 air           2584141.80  4132656.3  21766987.0  7842090.6   
emission_type2 water          317396.51  1254477.8   1012999.1  2449178.0   

region                            reg2              ...      reg5              \
sector                            food      mining  ... transport       other   
stressor       compartment                          ...                         
emission_type1 air          1697937.30  347378.150  ...  42299319  10773826.0   
emission_type2 water         204835.44   29463.944  ...   4199841   7191006.3   

region                            reg6                                       \
sector                            food     mining manufactoring electricity   
stressor       compartment                                                    
emission_type1 air          15777996.0  6420955.5   113172450.0  56022534.0   
emission_type2 water         4826108.1  1865625.1    12700193.0    753213.7   

region                                                                   
sector                     construction     trade   transport     other  
stressor       compartment                                               
emission_type1 air            4861838.5  18195621  47046542.0  21632868  
emission_type2 water          2699288.3  13892313   8765784.3  16782553  

[2 rows x 48 columns]

Theses index-names / columns-names need to match in order to match
characterization factors to the stressors.

The other columns names can be passed to the characterization method. By default the method assumes the following column names:

- impact: name of the characterization/impact
- factor: the numerical (float) multiplication value for a specific stressor to derive the impact/characterized account
- impact_unit: the unit of the calculated characterization/impact
- stressor_unit: the unit of the stressor in the extension

Alternative names can be passed through the parameters
*characterized_name_column*, *characterization_factors_column*, *characterized_unit_column* and *orig_unit_column*


Note, that the charact_table contains a characterization called 'total
emissions', for which the calculation requires a stressor not present in the
satellite account. This will be automatically omitted.

To calculate the characterization we use

In [18]:
char_emis = io.emissions.characterize(charact_table, name="impacts")

The parameter *name* is optional, if omitted the name will be set to
extension_name + _characterized. In case the passed name starts with an 
underscore, the return name with be the name of the original extension concatenated with the passed name.

The method call returns a namedTuple with *extension* and *factors*.
The attribute *extension* is the new extension which can be attached to the MRIO object.

In [19]:
print(char_emis.extension)

Extension impacts with parameters: name, F, F_Y, unit


The *factors* attribute contains information on the used characterization factors.

In [20]:
char_emis.factors

,stressor,compartment,impact,factor,impact_unit,stressor_unit,error_unit_impact,error_unit_stressor,error_missing_stressor,dropped
0,emission_type1,air,air water impact,0.002,t,kg,False,False,False,False
1,emission_type1,air,total emissions,1.000,kg,kg,False,False,False,False
2,emission_type1,air,total air emissions,0.001,t,kg,False,False,False,False
3,emission_type2,water,air water impact,0.001,t,kg,False,False,False,False
4,emission_type2,water,total emissions,1.000,kg,kg,False,False,False,False
5,emission_type3,land,total emissions,1.000,kg,kg,False,False,True,True


It reports on errors encountered during the processing (all columns starting with *error_*) and if a specific stressor was dropped. In the case above, emission_type3 is not available in the extension data, and the report specifies that this stressor was not used for the calculation. 

We can also choose to omit any impact which includes stressors not present in the extension. 
To do so, we set *drop_missing* to True:

In [21]:
char_emis_dropped = io.emissions.characterize(charact_table, name="impacts", drop_missing=True)

which results in 

In [22]:
char_emis_dropped.factors

,stressor,compartment,impact,factor,impact_unit,stressor_unit,error_unit_impact,error_unit_stressor,error_missing_stressor,dropped
0,emission_type1,air,air water impact,0.002,t,kg,False,False,False,False
1,emission_type1,air,total emissions,1.000,kg,kg,False,False,False,True
2,emission_type1,air,total air emissions,0.001,t,kg,False,False,False,False
3,emission_type2,water,air water impact,0.001,t,kg,False,False,False,False
4,emission_type2,water,total emissions,1.000,kg,kg,False,False,False,True
5,emission_type3,land,total emissions,1.000,kg,kg,False,False,True,True


All impacts effected by the missing stressor have been dropped. Compare

In [23]:
char_emis.extension.F

region                       reg1                                            \
sector                       food        mining manufactoring   electricity   
impact                                                                        
air water impact     3.835380e+03  1.995239e+03  4.799114e+04  5.655218e+04   
total air emissions  1.848065e+03  9.864481e+02  2.361379e+04  2.813910e+04   
total emissions      1.987315e+06  1.008791e+06  2.437736e+07  2.841308e+07   

region                                                                       \
sector               construction         trade     transport         other   
impact                                                                        
air water impact     5.485680e+03  9.519790e+03  4.454697e+04  1.813336e+04   
total air emissions  2.584142e+03  4.132656e+03  2.176699e+04  7.842091e+03   
total emissions      2.901538e+06  5.387134e+06  2.277999e+07  1.029127e+07   

region                       reg2                 ...          reg5  \
sector                       food         mining  ...     transport   
impact                                            ...                 
air water impact     3.600710e+03     724.220244  ...  8.879848e+04   
total air emissions  1.697937e+03     347.378150  ...  4.229932e+04   
total emissions      1.902773e+06  376842.094000  ...  4.649916e+07   

region                                     reg6                              \
sector                      other          food        mining manufactoring   
impact                                                                        
air water impact     2.873866e+04  3.638210e+04  1.470754e+04  2.390451e+05   
total air emissions  1.077383e+04  1.577800e+04  6.420956e+03  1.131724e+05   
total emissions      1.796483e+07  2.060410e+07  8.286581e+06  1.258726e+08   

region                                                                       \
sector                electricity  construction         trade     transport   
impact                                                                        
air water impact     1.127983e+05  1.242297e+04  5.028356e+04  1.028589e+05   
total air emissions  5.602253e+04  4.861838e+03  1.819562e+04  4.704654e+04   
total emissions      5.677575e+07  7.561127e+06  3.208793e+07  5.581233e+07   

region                             
sector                      other  
impact                             
air water impact     6.004829e+04  
total air emissions  2.163287e+04  
total emissions      3.841542e+07  

[3 rows x 48 columns]

with

In [24]:
char_emis_dropped.extension.F

region                     reg1                                          \
sector                     food       mining manufactoring  electricity   
impact                                                                    
air water impact     3835.38007  1995.239475   47991.14318  56552.18155   
total air emissions  1848.06480   986.448090   23613.78700  28139.10000   

region                                                               \
sector              construction      trade   transport       other   
impact                                                                
air water impact      5485.68011  9519.7904  44546.9731  18133.3592   
total air emissions   2584.14180  4132.6563  21766.9870   7842.0906   

region                     reg2              ...       reg5              \
sector                     food      mining  ...  transport       other   
impact                                       ...                          
air water impact     3600.71004  724.220244  ...  88798.479  28738.6583   
total air emissions  1697.93730  347.378150  ...  42299.319  10773.8260   

region                     reg6                                         \
sector                     food      mining manufactoring  electricity   
impact                                                                   
air water impact     36382.1001  14707.5361    239045.093  112798.2817   
total air emissions  15777.9960   6420.9555    113172.450   56022.5340   

region                                                               
sector              construction      trade    transport      other  
impact                                                               
air water impact      12422.9653  50283.555  102858.8683  60048.289  
total air emissions    4861.8385  18195.621   47046.5420  21632.868  

[2 rows x 48 columns]

Any/all of the extensions can be attached to the mrio system

In [25]:
io.impacts = char_emis.extension

and used for subsequent calculations:

In [26]:
io.calc_all()
io.impacts.D_cba

/home/konstans/proj/EXIOBASE/pymrio/pymrio/core/mriosystem.py:1074: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/home/konstans/proj/EXIOBASE/pymrio/pymrio/core/mriosystem.py:1158: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  self.D_cba.groupby(level="region", axis=1, sort=False).sum()
/home/konstans/proj/EXIOBASE/pymrio/pymrio/core/mriosystem.py:1167: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  self.D_pba.groupby(level="region", axis=1, sort=False).sum()
/home/konstans/proj/EXIOBASE/pymrio/pymrio/core/mriosystem.py:1175: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  self.D_imp_reg = self.D_imp.groupby(
/home/konstans/proj/EXIOBASE/pymrio/pymrio/core/mriosystem.py:1181: F

region                       reg1                                             \
sector                       food         mining manufactoring   electricity   
impact                                                                         
air water impact     4.354677e+03     384.125264  2.116984e+05  2.391231e+04   
total air emissions  2.056183e+03     179.423536  9.749300e+04  1.188759e+04   
total emissions      2.298494e+06  204701.727979  1.142054e+08  1.202472e+07   

region                                                                       \
sector               construction         trade     transport         other   
impact                                                                        
air water impact     7.032641e+03  8.548388e+03  2.200050e+04  4.012355e+04   
total air emissions  3.342906e+03  3.885884e+03  1.075027e+04  1.582152e+04   
total emissions      3.689735e+06  4.662504e+06  1.125023e+07  2.430203e+07   

region                       reg2                ...          reg5  \
sector                       food        mining  ...     transport   
impact                                           ...                 
air water impact     3.800328e+03     42.024811  ...  8.843384e+04   
total air emissions  1.793338e+03     19.145605  ...  4.209505e+04   
total emissions      2.006991e+06  22879.206385  ...  4.633879e+07   

region                                     reg6                              \
sector                      other          food        mining manufactoring   
impact                                                                        
air water impact     3.008044e+04  3.476528e+04  3.227857e+03  1.535829e+05   
total air emissions  1.138661e+04  1.517235e+04  1.345318e+03  7.145075e+04   
total emissions      1.869382e+07  1.959293e+07  1.882540e+06  8.213219e+07   

region                                                                       \
sector                electricity  construction         trade     transport   
impact                                                                        
air water impact     7.423616e+04  4.580343e+03  1.393218e+05  1.049447e+05   
total air emissions  3.683167e+04  1.836696e+03  4.241568e+04  4.805409e+04   
total emissions      3.740449e+07  2.743647e+06  9.690613e+07  5.689057e+07   

region                             
sector                      other  
impact                             
air water impact     1.183949e+05  
total air emissions  3.602298e+04  
total emissions      8.237196e+07  

[3 rows x 48 columns]

## Regional specific characterization factors

Here we use a table of regionally specific characterisation factors.
The actual factors contained here are the same as in the basic example and we
will modify them after loading. 
We will also investigate cases with missing data or conflicting units.
The same principles can be used for sector specific characterization factors.

We use the same data test mrio system as before: 

In [27]:
io = pymrio.load_test()

with the regional specific characterization factors from

In [28]:
charact_table_reg = pd.read_csv(
    (PYMRIO_PATH["test_mrio"] / Path("concordance") / "emissions_charact_reg_spec.tsv"),
    sep="\t",
)
charact_table_reg

,region,stressor,compartment,impact,factor,impact_unit,stressor_unit
0,reg1,emission_type1,air,air water impact,0.002,t,kg
1,reg1,emission_type2,water,air water impact,0.001,t,kg
2,reg1,emission_type1,air,total emissions,1.000,kg,kg
3,reg1,emission_type2,water,total emissions,1.000,kg,kg
4,reg1,emission_type3,land,total emissions,1.000,kg,kg
5,reg1,emission_type1,air,total air emissions,0.001,t,kg
6,reg2,emission_type1,air,air water impact,0.002,t,kg
7,reg2,emission_type2,water,air water impact,0.001,t,kg
8,reg2,emission_type1,air,total emissions,1.000,kg,kg
9,reg2,emission_type2,water,total emissions,1.000,kg,kg


Compared with the previous table (charact_table), this table contains an additional 
column *region* which contains the regional specific data.
Currently, the factors are actually the same as before, thus

In [29]:
char_reg = io.emissions.characterize(charact_table_reg)
char_reg.extension.F

region                       reg1                                            \
sector                       food        mining manufactoring   electricity   
impact                                                                        
air water impact     3.835380e+03  1.995239e+03  4.799114e+04  5.655218e+04   
total air emissions  1.848065e+03  9.864481e+02  2.361379e+04  2.813910e+04   
total emissions      1.987315e+06  1.008791e+06  2.437736e+07  2.841308e+07   

region                                                                       \
sector               construction         trade     transport         other   
impact                                                                        
air water impact     5.485680e+03  9.519790e+03  4.454697e+04  1.813336e+04   
total air emissions  2.584142e+03  4.132656e+03  2.176699e+04  7.842091e+03   
total emissions      2.901538e+06  5.387134e+06  2.277999e+07  1.029127e+07   

region                       reg2                 ...          reg5  \
sector                       food         mining  ...     transport   
impact                                            ...                 
air water impact     3.600710e+03     724.220244  ...  8.879848e+04   
total air emissions  1.697937e+03     347.378150  ...  4.229932e+04   
total emissions      1.902773e+06  376842.094000  ...  4.649916e+07   

region                                     reg6                              \
sector                      other          food        mining manufactoring   
impact                                                                        
air water impact     2.873866e+04  3.638210e+04  1.470754e+04  2.390451e+05   
total air emissions  1.077383e+04  1.577800e+04  6.420956e+03  1.131724e+05   
total emissions      1.796483e+07  2.060410e+07  8.286581e+06  1.258726e+08   

region                                                                       \
sector                electricity  construction         trade     transport   
impact                                                                        
air water impact     1.127983e+05  1.242297e+04  5.028356e+04  1.028589e+05   
total air emissions  5.602253e+04  4.861838e+03  1.819562e+04  4.704654e+04   
total emissions      5.677575e+07  7.561127e+06  3.208793e+07  5.581233e+07   

region                             
sector                      other  
impact                             
air water impact     6.004829e+04  
total air emissions  2.163287e+04  
total emissions      3.841542e+07  

[3 rows x 48 columns]

gives the same result as before. To highlight regional
specificity, we double the total emission factors of region 3.

In [30]:
charact_table_reg.loc[(charact_table_reg.region == "reg3") & (charact_table_reg.impact == "total emissions"), "factor"] = charact_table_reg.loc[(charact_table_reg.region == "reg3") & (charact_table_reg.impact == "total emissions"), "factor"] * 2

and calculate the new impacts

In [31]:
char_reg_dbl = io.emissions.characterize(charact_table_reg)
char_reg_dbl.extension.F.loc["total emissions"]

region  sector       
reg1    food             1.987315e+06
        mining           1.008791e+06
        manufactoring    2.437736e+07
        electricity      2.841308e+07
        construction     2.901538e+06
        trade            5.387134e+06
        transport        2.277999e+07
        other            1.029127e+07
reg2    food             1.902773e+06
        mining           3.768421e+05
        manufactoring    1.598022e+07
        electricity      1.660779e+07
        construction     1.868660e+06
        trade            3.511220e+06
        transport        6.836824e+06
        other            6.185187e+06
reg3    food             1.100035e+07
        mining           9.531717e+06
        manufactoring    2.150874e+08
        electricity      1.503010e+08
        construction     3.996900e+07
        trade            1.213563e+08
        transport        1.301629e+08
        other            3.714520e+08
reg4    food             6.479508e+06
        mining           9.5

compared to

In [32]:
char_reg.extension.F.loc["total emissions"]

region  sector       
reg1    food             1.987315e+06
        mining           1.008791e+06
        manufactoring    2.437736e+07
        electricity      2.841308e+07
        construction     2.901538e+06
        trade            5.387134e+06
        transport        2.277999e+07
        other            1.029127e+07
reg2    food             1.902773e+06
        mining           3.768421e+05
        manufactoring    1.598022e+07
        electricity      1.660779e+07
        construction     1.868660e+06
        trade            3.511220e+06
        transport        6.836824e+06
        other            6.185187e+06
reg3    food             5.500174e+06
        mining           4.765858e+06
        manufactoring    1.075437e+08
        electricity      7.515049e+07
        construction     1.998450e+07
        trade            6.067817e+07
        transport        6.508145e+07
        other            1.857260e+08
reg4    food             6.479508e+06
        mining           9.5

The return value also contains the report with the used factors

In [33]:
char_reg_dbl.factors

,stressor,compartment,region,impact,factor,impact_unit,stressor_unit,error_missing_region,error_unit_impact,error_unit_stressor,error_missing_stressor,dropped
0,emission_type1,air,reg1,air water impact,0.002,t,kg,False,False,False,False,False
1,emission_type1,air,reg1,total emissions,1.000,kg,kg,False,False,False,False,False
2,emission_type1,air,reg1,total air emissions,0.001,t,kg,False,False,False,False,False
3,emission_type1,air,reg2,air water impact,0.002,t,kg,False,False,False,False,False
4,emission_type1,air,reg2,total emissions,1.000,kg,kg,False,False,False,False,False
5,emission_type1,air,reg2,total air emissions,0.001,t,kg,False,False,False,False,False
6,emission_type1,air,reg3,air water impact,0.002,t,kg,False,False,False,False,False
7,emission_type1,air,reg3,total emissions,2.000,kg,kg,False,False,False,False,False
8,emission_type1,air,reg3,total air emissions,0.001,t,kg,False,False,False,False,False
9,emission_type1,air,reg4,air water impact,0.002,t,kg,False,False,False,False,False


Next, we will look at different cases of missing data handling.

## Missing data 

This section extends on the basic missing data section from the basic example above.
We will use the regional characterization table again, calling it *ch* to 
shorten the syntax.

In [34]:
ch = pd.read_csv(
    (PYMRIO_PATH["test_mrio"] / Path("concordance") / "emissions_charact_reg_spec.tsv"),
    sep="\t",
)
ch

,region,stressor,compartment,impact,factor,impact_unit,stressor_unit
0,reg1,emission_type1,air,air water impact,0.002,t,kg
1,reg1,emission_type2,water,air water impact,0.001,t,kg
2,reg1,emission_type1,air,total emissions,1.000,kg,kg
3,reg1,emission_type2,water,total emissions,1.000,kg,kg
4,reg1,emission_type3,land,total emissions,1.000,kg,kg
5,reg1,emission_type1,air,total air emissions,0.001,t,kg
6,reg2,emission_type1,air,air water impact,0.002,t,kg
7,reg2,emission_type2,water,air water impact,0.001,t,kg
8,reg2,emission_type1,air,total emissions,1.000,kg,kg
9,reg2,emission_type2,water,total emissions,1.000,kg,kg


And modify different rows to showcase missing/contradicting data.

First, we will cover the case with miss-aligned units.
Lets set the unit of the stressor *emission_type1* to "t".

In [35]:
ch.loc[ch.stressor == "emission_type1", "stressor_unit"] = "t"
ch

,region,stressor,compartment,impact,factor,impact_unit,stressor_unit
0,reg1,emission_type1,air,air water impact,0.002,t,t
1,reg1,emission_type2,water,air water impact,0.001,t,kg
2,reg1,emission_type1,air,total emissions,1.000,kg,t
3,reg1,emission_type2,water,total emissions,1.000,kg,kg
4,reg1,emission_type3,land,total emissions,1.000,kg,kg
5,reg1,emission_type1,air,total air emissions,0.001,t,t
6,reg2,emission_type1,air,air water impact,0.002,t,t
7,reg2,emission_type2,water,air water impact,0.001,t,kg
8,reg2,emission_type1,air,total emissions,1.000,kg,t
9,reg2,emission_type2,water,total emissions,1.000,kg,kg


Using this table for characterization leads to multiple dropped values, with 
the cause given in the column *error_unit_stressor*

In [36]:
unit_miss = io.emissions.characterize(ch)
unit_miss.factors

ERROR:root:Unit does not match extension unit for >('emission_type1', 'air')<


,stressor,compartment,region,impact,factor,impact_unit,stressor_unit,error_missing_region,error_unit_impact,error_unit_stressor,error_missing_stressor,dropped
0,emission_type1,air,reg1,air water impact,0.002,t,t,False,False,True,False,True
1,emission_type1,air,reg1,total emissions,1.000,kg,t,False,False,True,False,True
2,emission_type1,air,reg1,total air emissions,0.001,t,t,False,False,True,False,True
3,emission_type1,air,reg2,air water impact,0.002,t,t,False,False,True,False,True
4,emission_type1,air,reg2,total emissions,1.000,kg,t,False,False,True,False,True
5,emission_type1,air,reg2,total air emissions,0.001,t,t,False,False,True,False,True
6,emission_type1,air,reg3,air water impact,0.002,t,t,False,False,True,False,True
7,emission_type1,air,reg3,total emissions,1.000,kg,t,False,False,True,False,True
8,emission_type1,air,reg3,total air emissions,0.001,t,t,False,False,True,False,True
9,emission_type1,air,reg4,air water impact,0.002,t,t,False,False,True,False,True


Note: in production ready code one can guard against such cases by simply 
asserting that the error columns are False, e.g.

In [37]:
assert all(unit_miss.factors.error_unit_stressor is False), "Unit mismatch"

TypeError: 'bool' object is not iterable

Further note, since any error leads to dropping the specific row, one can also check for any dropped data to identify any error

### Characterizing calculated results
TODO: think of dropping this. Its trivial or dangerous, in case of regional characteriation

The characterize method can also be used to characterize already calculated
results. This works in the same way:

In [13]:
io_aly = pymrio.load_test().calc_all()

In [14]:
io_aly.emissions.D_cba

region                              reg1                               \
sector                              food         mining manufactoring   
stressor       compartment                                              
emission_type1 air          2.056183e+06  179423.535893  9.749300e+07   
emission_type2 water        2.423103e+05   25278.192086  1.671240e+07   

region                                                                \
sector                       electricity  construction         trade   
stressor       compartment                                             
emission_type1 air          1.188759e+07  3.342906e+06  3.885884e+06   
emission_type2 water        1.371303e+05  3.468292e+05  7.766205e+05   

region                                                          reg2  \
sector                         transport         other          food   
stressor       compartment                                             
emission_type1 air          1.075027e+07  1.582152e+07  1.793338e+06   
emission_type2 water        4.999628e+05  8.480505e+06  2.136528e+05   

region                                    ...          reg5                \
sector                            mining  ...     transport         other   
stressor       compartment                ...                               
emission_type1 air          19145.604911  ...  4.209505e+07  1.138661e+07   
emission_type2 water         3733.601474  ...  4.243738e+06  7.307208e+06   

region                              reg6                              \
sector                              food        mining manufactoring   
stressor       compartment                                             
emission_type1 air          1.517235e+07  1.345318e+06  7.145075e+07   
emission_type2 water        4.420574e+06  5.372216e+05  1.068144e+07   

region                                                                \
sector                       electricity  construction         trade   
stressor       compartment                                             
emission_type1 air          3.683167e+07  1.836696e+06  4.241568e+07   
emission_type2 water        5.728136e+05  9.069515e+05  5.449044e+07   

region                                                  
sector                         transport         other  
stressor       compartment                              
emission_type1 air          4.805409e+07  3.602298e+07  
emission_type2 water        8.836484e+06  4.634899e+07  

[2 rows x 48 columns]

In [15]:
io_aly.impacts = io_aly.emissions.characterize(charact_table, name="impacts_new")

Note, that all results which can be characterized directly (all flow accounts
like D_cba, D_pba, ...) are automatically included:

In [16]:
io_aly.impacts.D_cba

region                      reg1                                           \
sector                      food      mining  manufactoring   electricity   
impact                                                                      
air water impact     4354.677050  384.125264  211698.404833  23912.313854   
total air emissions  2056.183383  179.423536   97493.003893  11887.591799   

region                                                                     \
sector              construction        trade     transport         other   
impact                                                                      
air water impact     7032.640535  8548.387702  22000.497397  40123.553814   
total air emissions  3342.905658  3885.883601  10750.267281  15821.524280   

region                      reg2             ...          reg5                \
sector                      food     mining  ...     transport         other   
impact                                       ...                               
air water impact     3800.328439  42.024811  ...  88433.836812  30080.437653   
total air emissions  1793.337828  19.145605  ...  42095.049614  11386.614919   

region                       reg6                                            \
sector                       food       mining  manufactoring   electricity   
impact                                                                        
air water impact     34765.284482  3227.857425  153582.938963  74236.159458   
total air emissions  15172.354999  1345.317901   71450.748974  36831.672907   

region                                                                         
sector              construction          trade      transport          other  
impact                                                                         
air water impact     4580.343177  139321.814038  104944.664681  118394.939263  
total air emissions  1836.695857   42415.684852   48054.090102   36022.975074  

[2 rows x 48 columns]

Whereas coefficient accounts (M, S) are removed:

In [17]:
io_aly.impacts.M

To calculated these use

In [18]:
io_aly.calc_all()
io_aly.impacts.M

region                   reg1                                      \
sector                   food    mining manufactoring electricity   
impact                                                              
air water impact     0.022428  0.052572      0.000259    0.225083   
total air emissions  0.010865  0.025999      0.000127    0.111897   

region                                                              reg2  \
sector              construction     trade transport     other      food   
impact                                                                     
air water impact        0.000295  0.000092  0.000952  0.000291  0.000092   
total air emissions     0.000140  0.000042  0.000465  0.000132  0.000044   

region                         ...      reg5                reg6            \
sector                 mining  ... transport     other      food    mining   
impact                         ...                                           
air water impact     0.016122  ...  0.007839  0.000481  0.000976  0.018407   
total air emissions  0.007718  ...  0.003733  0.000182  0.000426  0.008250   

region                                                                \
sector              manufactoring electricity construction     trade   
impact                                                                 
air water impact         0.000949    0.447245     0.007421  0.003222   
total air emissions      0.000450    0.221901     0.002975  0.001207   

region                                   
sector              transport     other  
impact                                   
air water impact     0.001553  0.000739  
total air emissions  0.000711  0.000277  

[2 rows x 48 columns]

which will calculate the missing accounts.

For these calculations, the characterized accounts can also be used outside
the MRIO system. Thus:

In [19]:
independent_extension = io_aly.emissions.characterize(charact_table, name="impacts_new")

In [20]:
type(independent_extension)

pymrio.core.mriosystem.Extension

In [21]:
independent_extension.M

In [22]:
independent_extension_calc = independent_extension.calc_system(x=io_aly.x, Y=io_aly.Y)

In [23]:
independent_extension.M

region                   reg1                                      \
sector                   food    mining manufactoring electricity   
impact                                                              
air water impact     0.022428  0.052572      0.000259    0.225083   
total air emissions  0.010865  0.025999      0.000127    0.111897   

region                                                              reg2  \
sector              construction     trade transport     other      food   
impact                                                                     
air water impact        0.000295  0.000092  0.000952  0.000291  0.000092   
total air emissions     0.000140  0.000042  0.000465  0.000132  0.000044   

region                         ...      reg5                reg6            \
sector                 mining  ... transport     other      food    mining   
impact                         ...                                           
air water impact     0.016122  ...  0.007839  0.000481  0.000976  0.018407   
total air emissions  0.007718  ...  0.003733  0.000182  0.000426  0.008250   

region                                                                \
sector              manufactoring electricity construction     trade   
impact                                                                 
air water impact         0.000949    0.447245     0.007421  0.003222   
total air emissions      0.000450    0.221901     0.002975  0.001207   

region                                   
sector              transport     other  
impact                                   
air water impact     0.001553  0.000739  
total air emissions  0.000711  0.000277  

[2 rows x 48 columns]

## Inspecting the used characterization table

Pymrio automatically adjust the characterization table by removing accounts
which can not be calculated using a given extension. The removed accounts are
reported through a warning message (e.g. "WARNING:root:Impact >total
emissions< removed - calculation requires stressors not present in extension
>Emissions<" in the examples above).

It is also possible, to obtain the cleaned characterization-table for
inspection and further use. To do so:

In [24]:
impacts = io.emissions.characterize(
    charact_table, name="impacts", return_char_matrix=True
)

This changes the return type from a pymrio.Extension to a named tuple

In [25]:
type(impacts)

pymrio.core.mriosystem.characterization

with

In [26]:
impacts.extension

and

In [27]:
impacts.factors

,stressor,compartment,impact,factor,impact_unit
0,emission_type1,air,air water impact,0.002,t
1,emission_type2,water,air water impact,0.001,t
5,emission_type1,air,total air emissions,0.001,t


The latter is the characterization table used for the calculation.

For further information see the characterization docstring:

In [28]:
print(io.emissions.characterize.__doc__)

Characterize stressors

        Characterizes the extension with the characterization factors given in factors.
        Factors can contain more characterization factors which depend on stressors not
        present in the Extension - these will be automatically removed.

        Note
        ----
        Accordance of units is not checked - you must ensure that the
        characterization factors correspond to the units of the extension to be
        characterized.

        Parameters
        -----------
        factors: pd.DataFrame
            A dataframe in long format with numerical index and columns named
            index.names of the extension to be characterized and
            'characterized_name_column', 'characterization_factors_column',
            'characterized_unit_column'

        characterized_name_column: str (optional)
            Name of the column with the names of the
            characterized account (default: "impact")

        characterization_factors_column: